In [98]:
import pandas as pd
import numpy as np
import math
import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error

from scipy.fftpack import fft
from scipy.stats import iqr
from scipy import signal
# import matplotlib.pyplot as plt


In [63]:
insulin_data = pd.read_csv('InsulinData.csv', parse_dates=[['Date','Time']], keep_date_col=True, low_memory=False)
insulin_df = insulin_data[['Date_Time', 'Index', 'BWZ Carb Input (grams)']]
insulin_df.loc[:, 'Index']

glucose_data = pd.read_csv('CGMData.csv', parse_dates=[['Date','Time']], keep_date_col=True, low_memory=False)
glucose_df = glucose_data[['Date_Time', 'Sensor Glucose (mg/dL)']]

In [ ]:

def choose_bin(x, min_carb, total_bins):
    partition = float((x - min_carb)/20)
    bin =  math.floor(partition)
    if bin == total_bins:
        bin = bin - 1
    return bin

meals = []
meals_df = pd.DataFrame()
meal_matrix = pd.DataFrame()
two_hours = 60 * 60 * 2
thirty_min = 30 * 60
sensor_time_interval = 30

bin_matrix = []
bins = []
min_carb = 0
max_carb = 0
total_bins = 0

processed_insulin_df = insulin_df.copy()
processed_glucose_df = glucose_df.copy()

# process insulin data
valid_carb_input = processed_insulin_df['BWZ Carb Input (grams)'].notna() & processed_insulin_df['BWZ Carb Input (grams)'] != 0.0
processed_insulin_df = processed_insulin_df.loc[valid_carb_input][['Date_Time', 'BWZ Carb Input (grams)']]
processed_insulin_df.set_index(['Date_Time'], inplace = True)
processed_insulin_df = processed_insulin_df.sort_index().reset_index()

valid_glucose = processed_glucose_df['Sensor Glucose (mg/dL)'].notna()
processed_glucose_df = processed_glucose_df.loc[valid_glucose][['Date_Time', 'Sensor Glucose (mg/dL)']]
processed_glucose_df.set_index(['Date_Time'], inplace = True)
processed_glucose_df = processed_glucose_df.sort_index().reset_index()

min_carb = processed_insulin_df['BWZ Carb Input (grams)'].min()
max_carb = processed_insulin_df['BWZ Carb Input (grams)'].max()
total_bins = math.ceil((max_carb - min_carb) / 20)

for i in range(len(processed_insulin_df)):
    carb_input = processed_insulin_df['BWZ Carb Input (grams)'][i]
    selected_bin = choose_bin(carb_input, min_carb, total_bins)
    bins.append(selected_bin)

processed_insulin_df['bin'] = bins

for i in range(0, len(processed_insulin_df)-1):
    time_diff_seconds = (processed_insulin_df.iloc[i + 1]['Date_Time'] - processed_insulin_df.iloc[i]['Date_Time']).total_seconds()
    if(time_diff_seconds > two_hours):
        meals.append(True)
    else:
        meals.append(False)
    
meals.append(True)
meals_df = processed_insulin_df[meals]

for i in range(len(meals_df)):
    lower_bound = meals_df.iloc[i]['Date_Time'] - datetime.timedelta(seconds=thirty_min)
    upper_bound = meals_df.iloc[i]['Date_Time'] + datetime.timedelta(seconds=two_hours)
    is_within_bounds = (processed_glucose_df['Date_Time'] >= lower_bound) & (processed_glucose_df['Date_Time'] < upper_bound)
    bin = meals_df.iloc[i]['bin']
    filtered_glucose_df = processed_glucose_df[is_within_bounds]
    
    if len(filtered_glucose_df.index) == sensor_time_interval:
        filtered_glucose_df = filtered_glucose_df.T
        filtered_glucose_df.drop('Date_Time', inplace=True)
        
        filtered_glucose_df.reset_index(drop=True, inplace=True)
        filtered_glucose_df.columns = list(range(1, 31))
        
        meal_matrix = meal_matrix.append(filtered_glucose_df, ignore_index=True)
        bin_matrix.append(bin)

meal_matrix = meal_matrix.apply(pd.to_numeric)
bin_matrix = np.array(bin_matrix)

In [78]:
def calc_abs_mean(df_row):
    abs_mean = 0
    for i in range(0, len(df_row) - 1):
        abs_mean = abs_mean + np.abs(df_row[(i + 1)] - df_row[i])
    return abs_mean / len(df_row)


def entropy(df_row):
    entropy = 0
    if (len(df_row) <= 1):
        return 0
    else:
        value, index = np.unique(df_row, return_counts=True)
        ratio = index / len(df_row)
        non_zero_ratio = np.count_nonzero(ratio)

        if non_zero_ratio <= 1:
            return 0
        for i in ratio:
            entropy -= i * np.log2(i)
        return entropy

def calculate_psd(row):
    f,power_values=signal.periodogram(row)
    psd1=power_values[:5].mean()
    psd2=power_values[5:10].mean()
    psd3=power_values[10:16].mean()
    return psd1,psd2,psd3

def calculate_fft(row):
    # Compute the FFT of 'x'
    Pxx = signal.periodogram(row)
    fft_x = fft(row)
    # Compute the power spectrum of the FFT
    power_spectrum = np.abs(fft_x)**2
    # Sort the power spectrum in descending order
    sorted_indices = np.argsort(power_spectrum)[::-1]
    # Ignore the 1st peak
    sorted_indices = sorted_indices[1:]
    # Consider the 2nd to 7th peak's power only
    selected_indices = sorted_indices[:6]
    new_power=[]
    for ind in selected_indices:
        new_power.append(power_spectrum[ind])
    return new_power

In [99]:

features = pd.DataFrame()

for i in range(0, meal_matrix.shape[0]):
    x = meal_matrix.iloc[i, :].tolist()
    
    fft_powerValues=calculate_fft(x)
    psd1,psd2,psd3=calculate_psd(x)
    features = features.append({

        "VelocityMin": np.diff(x).min(),
        "VelocityMax": np.diff(x).max(),
        "VelocityMean": np.diff(x).mean(),
        "AccelerationMin":np.diff(np.diff(x)).min(),
        "AccelerationMax":np.diff(np.diff(x)).max(),
        "AccelerationMean":np.diff(np.diff(x)).mean(),
        "Entropy": entropy(x),
        "iqr":iqr(x),
        "fft1": fft_powerValues[0],
        "fft2": fft_powerValues[1],
        "fft3": fft_powerValues[2],
        "fft4": fft_powerValues[3],
        "fft5": fft_powerValues[4],
        "fft6": fft_powerValues[5],
        "psd1":psd1,
        "psd2":psd2,
        "psd3":psd3
    },
    ignore_index=True
    )

feature_matrix = features.to_numpy()
#print('feature matrix: ', np.shape(feature_matrix))


scaler = StandardScaler()
scaled_features = scaler.fit_transform(feature_matrix)
print('scaled_features: ', scaled_features)
# scaled = pd.DataFrame(scaled_features)
# scaled.to_csv('scaled.csv')


<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipyt

scaled_features:  [[ 0.06318191 -0.63614385 -0.54584316 ... -0.60026781 -0.55354284
  -0.54419754]
 [-1.19498599  0.1677742   0.21765354 ... -0.52459092 -0.5154497
  -0.47883771]
 [ 0.81808265 -0.63614385 -0.28374728 ... -0.81224993 -0.6297665
  -0.64234772]
 ...
 [ 0.06318191  0.1677742  -0.26095633 ... -0.8605243  -0.56761624
  -0.58023071]
 [-0.44008525  0.97169225  0.55951773 ... -0.43088203 -0.32039001
  -0.25257996]
 [ 0.1889987  -1.21037103 -0.24956086 ... -0.8509513  -0.62622663
  -0.6277821 ]]


<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipython-input-99-62c41c755bff>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append({
<ipyt

In [100]:


def ground_truth_cm(k, clusters, ground_truth):
    cluster_matrix = np.zeros((k, k))
    for i, j in enumerate(ground_truth):
        row = clusters[i]
        col = j
        cluster_matrix[row][col] += 1
    return cluster_matrix

def calc_dbscan_sse(labels, feature_matrix):
    sum = 0
    cluster_size = max(labels)
    for i in range(cluster_size + 1):
        x = feature_matrix[labels == i] - feature_matrix[labels == i].mean(axis=0) 
        sum = np.sum(x ** 2)
    return sum

def calc_cluster_entropy(gtm):
    gtm_sum = gtm.sum()
    bins = gtm.shape[0]
    cluster_entropy = 0
    cluster_sum = 0
    cluster_entropies = []

    for i in range(bins):
        cluster_sum = np.sum(gtm[i])
        if cluster_sum == 0:
            continue
        for j in range(bins):
            if gtm[i,j] == 0:
                continue
            col_sum = gtm[i,j] / cluster_sum
            entropy = -1 * col_sum * np.log2(col_sum)
            cluster_entropy = cluster_entropy + entropy
        cluster_entropies.append((cluster_sum / gtm_sum) * cluster_entropy)
    return np.sum(cluster_entropies)

def calc_cluster_purity(gtm):
    gtm_sum = gtm.sum()
    bins = gtm.shape[0]
    cluster_sum = 0
    cluster_purity = 0
    cluster_max = 0
    cluster_purities = []

    for i in range(bins):
        cluster_max = np.max(gtm[i])
        cluster_sum = np.sum(gtm[i])
        if cluster_sum == 0:
            continue
        cluster_purity = cluster_max / cluster_sum
        cluster_purities.append((cluster_sum / gtm_sum) * cluster_purity)
    return np.sum(cluster_purities)


# calculate k means
kmeans = KMeans(n_clusters=total_bins, random_state=0).fit(scaled_features)
kmeans_centroid_locations = kmeans.cluster_centers_
kmeans_labels = kmeans.labels_
kmeans_gtm = ground_truth_cm(int(total_bins), kmeans_labels, bin_matrix)
kmeans_sse = kmeans.inertia_
kmeans_entropy = calc_cluster_entropy(kmeans_gtm)
kmeans_purity = calc_cluster_purity(kmeans_gtm)

# calculate dbscan 
default_epsilon = 50 # retrieved by observing plotted euclidian distances
dbscan = DBSCAN(eps=default_epsilon, min_samples=total_bins, metric="euclidean").fit(feature_matrix)
dbscan_labels = dbscan.labels_
dbscan_clusters = len(np.unique(dbscan_labels))
dbscan_outliers = np.sum(np.array(dbscan_labels) == -1, axis=0)
dbscan_gtm = ground_truth_cm(int(total_bins), dbscan_labels, bin_matrix)


dbscan_sse = calc_dbscan_sse(dbscan_labels, scaled_features)
dbscan_entropy = calc_cluster_entropy(dbscan_gtm)
dbscan_purity = calc_cluster_purity(dbscan_gtm)

/Users/wingman2.0/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [101]:
output = pd.DataFrame(
    [
        [
            kmeans_sse,
            dbscan_sse,
            kmeans_entropy,
            dbscan_entropy,
            kmeans_purity,
            dbscan_purity,
        ]
    ],
    columns=[
        "SSE for KMeans",
        "SSE for DBSCAN",
        "Entropy for KMeans",
        "Entropy for DBSCAN",
        "Purity for KMeans",
        "Purity for DBSCAN",
    ],
)
output = output.fillna(0)
output.to_csv("Results.csv", index=False, header=None)